In [9]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs
!pip install torch numpy scipy soundfile ffmpeg-python PySoundFile tqdm yt_dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-r42_wafj/demucs_bd63ce96d5214ad1a0dbe21cf253e201
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-r42_wafj/demucs_bd63ce96d5214ad1a0dbe21cf253e201
  Resolved https://github.com/facebookresearch/demucs to commit 8b48c27f82b0e119c91c613eee2ce87ba28575a0
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import yt_dlp
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO
from google.colab import files
import zipfile
import os


In [11]:
# Customize the following options!
model = "htdemucs_ft"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = '/content/demucs'
out_path = '/content/demucs_separated'

In [12]:

# Prompt the user to enter a YouTube URL
url = input("Enter a YouTube URL to download audio from: ")

# Create output directory if it doesn't exist
Path(out_path).mkdir(parents=True, exist_ok=True)

# Download audio from the entered URL and save to the input directory
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': os.path.join(in_path, '%(title)s.%(ext)s'),
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '320'
    }]
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

Enter a YouTube URL to download audio from: https://www.youtube.com/watch?v=MAabzMP5WSg
[youtube] Extracting URL: https://www.youtube.com/watch?v=MAabzMP5WSg
[youtube] MAabzMP5WSg: Downloading webpage
[youtube] MAabzMP5WSg: Downloading android player API JSON
[info] MAabzMP5WSg: Downloading 1 format(s): 251
[dashsegments] Total fragments: 1
[download] Destination: /content/demucs/Impellitteri - Venom (Official ⧸ New ⧸ Studio Album ⧸ 2015).webm
[download] 100% of    3.49MiB in 00:00:00 at 12.37MiB/s              
[ExtractAudio] Destination: /content/demucs/Impellitteri - Venom (Official ⧸ New ⧸ Studio Album ⧸ 2015).mp3
Deleting original file /content/demucs/Impellitteri - Venom (Official ⧸ New ⧸ Studio Album ⧸ 2015).webm (pass -k to keep)


In [13]:

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')
    
    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()
    
    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()
    
    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)

In [ ]:
separate()

Going to separate the files:
/content/demucs/Impellitteri - Venom (Official ⧸ New ⧸ Studio Album ⧸ 2015).mp3
With command:  python3 -m demucs.separate -o /content/demucs_separated -n htdemucs_ft --mp3 --mp3-bitrate=320


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/f7e0c4bc-ba3fe64a.th" to /root/.cache/torch/hub/checkpoints/f7e0c4bc-ba3fe64a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 150MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/d12395a8-e57c48e6.th" to /root/.cache/torch/hub/checkpoints/d12395a8-e57c48e6.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 149MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/92cfc3b6-ef3bcb9c.th" to /root/.cache/torch/hub/checkpoints/92cfc3b6-ef3bcb9c.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 119MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/04573f0d-f3cf25b2.th" to /root/.cache/torch/hub/checkpoints/04573f0d-f3cf25b2.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 130MB/s]


In [ ]:



# Download
from google.colab import files
files.download("/content/demucs_separated/htdemucs_ft")